In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.metrics import f1_score
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.regularizers import l2
from keras import utils

In [2]:
# for local
# processed_directory = '../processed_data/'

# for google drive
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    path_to_file = '/content/gdrive/My Drive/cs4248-project/'
    print(path_to_file)
    # move to Google Drive directory
    os.chdir(path_to_file)
    !pwd

Mounted at /content/gdrive
/content/gdrive/My Drive/cs4248-project/
/content/gdrive/My Drive/cs4248-project


In [7]:
data_path = '/content/gdrive/My Drive/cs4248-project/processed_data'
train_df = pd.read_json(data_path+'/train.json')
test_df = pd.read_json(data_path+'/test.json')

In [8]:
train_df.head()

,id,text,rating,label
0,0,Bromwell High is a cartoon comedy. It ran at t...,9,+
1,10000,Homelessness (or Houselessness as George Carli...,8,+
2,10001,Brilliant over-acting by Lesley Ann Warren. Be...,10,+
3,10002,This is easily the most underrated film inn th...,7,+
4,10003,This is not the typical Mel Brooks film. It wa...,8,+


In [9]:
test_df.head()

,id,text,rating,label
0,0,I went and saw this movie last night after bei...,10,+
1,10000,Actor turned director Bill Paxton follows up h...,7,+
2,10001,As a recreational golfer with some knowledge o...,9,+
3,10002,"I saw this film in a sneak preview, and it is ...",8,+
4,10003,Bill Paxton has taken the true story of the 19...,8,+


In [10]:
mapping = {'+': 0, '-': -1}

train_x = train_df["text"]
train_y = train_df.replace({'label': mapping})
train_y = train_y["label"]

test_x = test_df["text"]
test_y = test_df.replace({'label': mapping})
test_y = test_y["label"]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_x)

In [11]:
train_y

0        0
1        0
2        0
3        0
4        0
        ..
24995   -1
24996   -1
24997   -1
24998   -1
24999   -1
Name: label, Length: 25000, dtype: int64

In [12]:
sequences = tokenizer.texts_to_sequences(train_x)

In [13]:
# pad sequence to longest sequence
train_x_arr = [sentence.split() for sentence in train_x]
longest_seq = max(len(arr) for arr in train_x_arr)

print(longest_seq)

2470


In [14]:
num_classes = 2

X_train = pad_sequences(sequences, maxlen=longest_seq, padding='post')
Y_train = tf.keras.utils.to_categorical(train_y, num_classes)

In [15]:
X_train

array([[21905,   309,     6, ...,     0,     0,     0],
       [23379,    39, 52009, ...,     0,     0,     0],
       [  526,   117,   113, ...,     0,     0,     0],
       ...,
       [   10,   216, 88578, ...,     0,     0,     0],
       [   46,   105,    12, ...,     0,     0,     0],
       [   11,     6,    28, ...,     0,     0,     0]], dtype=int32)

In [16]:
Y_train

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [17]:
sequences_test = tokenizer.texts_to_sequences(test_x)

X_test = pad_sequences(sequences_test, maxlen=longest_seq, padding='post')
Y_test = tf.keras.utils.to_categorical(test_y, num_classes)

In [18]:
vocab_size = len(tokenizer.word_index) + 1

In [19]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=longest_seq))
model.add(Conv1D(filters=16, kernel_size=4, activation="relu"))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation="relu"))
model.add(Dense(2, activation="softmax"))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2470, 100)         8858300   
                                                                 
 conv1d (Conv1D)             (None, 2467, 16)          6416      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1233, 16)         0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 19728)             0         
                                                                 
 dense (Dense)               (None, 10)                197290    
                                                                 
 dense_1 (Dense)             (None, 2)                 22        
                                                        

In [23]:
history = model.fit(X_train, Y_train, epochs=9, batch_size=20)

Epoch 1/9
1250/1250 [==============================] - 26s 21ms/step - loss: 0.0078 - accuracy: 0.9980
Epoch 2/9
1250/1250 [==============================] - 26s 21ms/step - loss: 0.0028 - accuracy: 0.9992
Epoch 3/9
1250/1250 [==============================] - 26s 21ms/step - loss: 3.3555e-04 - accuracy: 0.9999
Epoch 4/9
1250/1250 [==============================] - 26s 21ms/step - loss: 7.1756e-05 - accuracy: 1.0000
Epoch 5/9
1250/1250 [==============================] - 26s 21ms/step - loss: 5.6399e-05 - accuracy: 1.0000
Epoch 6/9
1250/1250 [==============================] - 26s 21ms/step - loss: 5.2198e-05 - accuracy: 1.0000
Epoch 7/9
1250/1250 [==============================] - 26s 21ms/step - loss: 3.7006e-06 - accuracy: 1.0000
Epoch 8/9
1250/1250 [==============================] - 26s 21ms/step - loss: 1.4346e-06 - accuracy: 1.0000
Epoch 9/9
1250/1250 [==============================] - 26s 21ms/step - loss: 6.9628e-07 - accuracy: 1.0000


In [24]:
loss, accuracy = model.evaluate(X_test, Y_test)

print('Test Accuracy: %f' % (accuracy*100))

782/782 [==============================] - 7s 8ms/step - loss: 1.4884 - accuracy: 0.8559
Test Accuracy: 85.592002


In [25]:
def get_verdict(x):
    if x == 'A':
        return 0
    else:
        return -1

y_pred = model.predict(X_test)
df = pd.DataFrame(y_pred, columns = ['A', 'B'])

y_pred = pd.DataFrame.idxmax(df, axis=1)
y_pred = y_pred.apply(lambda x: get_verdict(x)).to_frame()

y_pred = y_pred.to_numpy().flatten()

y_pred

score = f1_score(test_y, y_pred, average='macro')
print('f1 score = {}'.format(score))

f1 score = 0.8558753559502592


In [ ]:
# f1 score = 0.8558753559502592